<a href="https://colab.research.google.com/github/tiffchu/spotify-sandbox/blob/main/spotify_apr14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "7502c71b4d294aa58286f5e3a2c6dd74"
client_secret = "72177ee7e0cf481895fe5146619dfe45"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import json
import numpy as np

def get_token():
  auth_string = client_id + ":" + client_secret
  auth_bytes = auth_string.encode("utf-8")
  auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
  url = 'https://accounts.spotify.com/api/token'
  headers = {
      "authorization" : "Basic " + auth_base64,
      "content_type": "application/x-www-urlencoded"
  }
  data = {"grant_type": "client_credentials"}
  result = post(url, headers=headers, data=data)
  json_result = json.loads(result.content)
  token = json_result["access_token"]
  return token

token = get_token()
print(token)

BQCmkIpxSC7x-qCcF-hdDdRjIqhA0XexZHwboRMcLauo28pe7JyecWBGotYxshVBk-peFcGuj4-dAlQeAjo5qbEtCnYZU1C3xO2NyTnOtlmCMqgK-pc


In [ ]:
import requests

token_url = 'https://accounts.spotify.com/api/token'

#payload for the POST request
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Make the POST request to obtain the access token
response = requests.post(token_url, data=payload)

# Parse the JSON response
response_data = response.json()

# Extract the access token from the response
access_token = response_data['access_token']

# Print the access token
print("Access Token:", access_token)


Access Token: BQB57hy6qQhcfr26QWyMJ6zrQ0F807jiZLK5FFhOo1ynghlKpUwgq0DI9aKMvesFFy2aujPvAfSWk7LS4hE3xB_K0fUskCHSTmWrcUcmHRzVuBtNp5A


In [ ]:
name = ["sleepy fish"]
result = sp.search(name)
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1IJe80moz409PtxW4llPFw'},
  'href': 'https://api.spotify.com/v1/artists/1IJe80moz409PtxW4llPFw',
  'id': '1IJe80moz409PtxW4llPFw',
  'name': 'Sleepy Fish',
  'type': 'artist',
  'uri': 'spotify:artist:1IJe80moz409PtxW4llPFw'}]

In [ ]:
porter = sp.search(q='artist:Future', limit=50)
for track in porter['tracks']['items']:
    print(track['name'], '-', track['artists'][0]['name'])

Like That - Future
Type Shit - Future
Cinderella - Future
We Don't Trust You - Future
Young Metro - Future
We Still Don't Trust You - Future
Solo - Future
Ice Attack - Future
WAIT FOR U (feat. Drake & Tems) - Future
PUFFIN ON ZOOTIEZ - Future
Everyday Hustle - Future
Slimed In - Future
Low Life (feat. The Weeknd) - Future
Red Leather - Future
Fried (She a Vibe) - Future
Out Of My Hands - Future
Mask Off - Future
Magic Don Juan (Princess Diana) - Future
Show of Hands - Future
Claustrophobic - Future
Runnin Outta Time - Future
All My Life - Future
Drink N Dance - Future
GTA - Future
Ain't No Love - Future
Jealous - Future
All to Myself - Future
The Percocet & Stripper Joint - Future
Nights Like This - Future
WTFYM - Future
Streets Made Me A King - Future
Luv Bad Bitches - Future
Seen it All - Future
This Sunday - Future
Where My Twin @ - Bonus - Future
Crossed Out - Future
Nobody Knows My Struggle - Future
Crazy Clientele - Future
Beat It - Future
Amazing (Interlude) - Future
LOVE YOU BE

In [ ]:
results = sp.search(q='artist:Future', limit=50)

for track in results['tracks']['items']:
    print(track['name'], '-', track['artists'][0]['name'])
    track_id = track['id']

    # audio features only if track_id is not None
    if track_id:
        audio_features = sp.audio_features([track_id])
        if audio_features is not None and audio_features[0] is not None:
            print("Audio Features:")
            for feature, value in audio_features[0].items():
                print(f"{feature}: {value}")
        else:
            print("Audio features not available for this track.")
    else:
        print("Track ID is None.")


Like That - Future


ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2tudvzsrR56uom6smgOcSf:
 Max Retries, reason: too many 429 error responses

In [ ]:
import pandas as pd

artist_name = 'Future'

# Perform a search for the artist
results = sp.search(q='artist:' + artist_name, type='artist')

# Get the artist ID
artist_id = results['artists']['items'][0]['id']

# Initialize lists to store track information and audio features
tracks_info = []
audio_features_list = []

# Fetch the artist's top tracks
top_tracks = sp.artist_top_tracks(artist_id)

# Iterate over the top tracks
for track in top_tracks['tracks']:
    # Get track information
    track_info = {
        'Track Name': track['name'],
        'Artist Name': artist_name,
        'Track ID': track['id']
    }
    tracks_info.append(track_info)

    # Get audio features for the track
    audio_features = sp.audio_features([track['id']])
    if audio_features and audio_features[0]:
        audio_features_list.append(audio_features[0])
    else:
        # If audio features are not available, append None
        audio_features_list.append(None)

# Create DataFrames from the lists of track information and audio features
tracks_df = pd.DataFrame(tracks_info)
audio_features_df = pd.DataFrame(audio_features_list)

# Merge the two DataFrames on the index (outer join to include all tracks)
pd.concat([tracks_df, audio_features_df], axis=1)


ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2tudvzsrR56uom6smgOcSf:
 Max Retries, reason: too many 429 error responses

In [ ]:
# REFER TO THIS

artist_name = 'Issy Wood'

# Search for the artist
results = sp.search(q='artist:' + artist_name, type='artist')

# Get the artist ID
artist_id = results['artists']['items'][0]['id']

# Get albums from the artist
albums = sp.artist_albums(artist_id, album_type='album', limit=50)

# Create a list to store track information and audio features
tracks_info = []
audio_features_list = []

# Iterate over each album
for album in albums['items']:
    # Get the album ID
    album_id = album['id']

    # Get tracks from the album
    tracks = sp.album_tracks(album_id)

    # Iterate over each track
    for track in tracks['items']:
        # Get track information
        track_info = {
            'Track Name': track['name'],
            'Album Name': album['name'],
            'Release Date': album['release_date'],
        }
        tracks_info.append(track_info)

        # Get audio features for the track
        audio_features = sp.audio_features([track['id']])[0]
        audio_features_list.append(audio_features)

# Create DataFrames from the lists of track information and audio features
tracks_df = pd.DataFrame(tracks_info)
audio_features_df = pd.DataFrame(audio_features_list)

# Merge the two DataFrames on index
pd.concat([tracks_df, audio_features_df], axis=1)

ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=6xctk1HS4sVso6wP8BgdeK:
 Max Retries, reason: too many 429 error responses

In [ ]:
#works but error reason: too many 429 error responses

import time

start_time = time.time()
artist_names = ['Drake', 'J Cole', 'Travis Scott', 'Megan Thee Stallion', 'Doja Cat',
                'Kid Cudi', 'Nicki Minaj', '21 Savage', 'Metro Boomin', 'Gunna',
                'Kendrick Lamar', 'Jack Harlow', 'Eminem', 'Tupac', 'Biggie', 'NWA']

all_tracks_info = []
all_audio_features = []

# Loop through each artist
for artist_name in artist_names:
    # Search for the artist
    results = sp.search(q='artist:' + artist_name, type='artist')
    artist_id = results['artists']['items'][0]['id']

    # Get albums from the artist
    albums = sp.artist_albums(artist_id, album_type='album', limit=50)

    # Iterate over each album
    for album in albums['items']:
        # Get the album ID
        album_id = album['id']

        # Get tracks from the album
        tracks = sp.album_tracks(album_id)

        # Iterate over each track
        for track in tracks['items']:
            # Get track information
            track_info = {
                'Artist Name': artist_name,
                'Track Name': track['name'],
                'Album Name': album['name'],
                'Release Date': album['release_date'],
            }
            all_tracks_info.append(track_info)

            # Get audio features for the track
            audio_features = sp.audio_features([track['id']])[0]
            all_audio_features.append(audio_features)

# Create DataFrames from the lists of track information and audio features
tracks_df = pd.DataFrame(all_tracks_info)
audio_features_df = pd.DataFrame(all_audio_features)

# Merge the two DataFrames on index
pd.concat([tracks_df, audio_features_df], axis=1)

end_time = time.time()

# Calculate runtime
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")

ERROR:spotipy.client:Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=7IqsNN0EMckx9YqddgXlwg:
 Max Retries, reason: too many 429 error responses

In [ ]:
#function - audio features from album url
def extract_album_id(album_url):
    album_id = album_url.split('/')[-1].split('?')[0]
    return album_id

def get_audio_features(track_id):
    audio_features = sp.audio_features([track_id])[0]
    return audio_features

def get_album(album_url):
    album_id = extract_album_id(album_url)
    album_info = sp.album(album_id)
    return album_info

#baseball sound effects album
album_url = 'https://open.spotify.com/album/3buFPZgdX6l89c1qycFlJN?si=tqhJy9jCQSGDPsA02vtZFQ'
album_info = get_album(album_url)

#initalize empty df
track_names = []
audio_features_list = []

for track in album_info['tracks']['items']:
    track_name = track['name']
    track_id = track['id']
    print("\nTrack Name:", track_name)

    audio_features = get_audio_features(track_id)
    print("Audio Features:", audio_features)

    #APPEND TO DF
    track_names.append(track_name)
    audio_features_list.append(audio_features)

pd.DataFrame(audio_features_list, index=track_names)
#download below csv
   df = pd.DataFrame(audio_features_list, index=track_names)
   df.to_csv('album_audio_features.csv', index=True)

In [ ]:
# Function to extract playlist ID from Spotify playlist URL
def extract_playlist_id(playlist_url):
    playlist_id = playlist_url.split('/')[-1].split('?')[0]
    return playlist_id

# Function to get audio features for a track
def get_audio_features(track_id):
    audio_features = sp.audio_features([track_id])[0]
    return audio_features

# Function to get playlist information
def get_playlist(playlist_url):
    playlist_id = extract_playlist_id(playlist_url)
    playlist_info = sp.playlist(playlist_id)
    return playlist_info

#inputs
playlist_url = 'https://open.spotify.com/playlist/37i9dQZEVXbLRQDuF5jeBp?si=0d45b0b6067e404d' #top 50 USA
playlist_info = get_playlist(playlist_url)
print("Playlist Name:", playlist_info['name'])
print("Playlist Owner:", playlist_info['owner']['display_name'])
print("Number of Tracks:", playlist_info['tracks']['total'])

# Iterate over tracks in the playlist and get audio features for each track
num_tracks_to_process = 10  # Set the limit here
for index, item in enumerate(playlist_info['tracks']['items']):
    if index >= num_tracks_to_process:
        break

    track = item['track']
    print("Track Name:", track['name'])
    print("Artist(s):", ", ".join([artist['name'] for artist in track['artists']]))

    # Get audio features for the track
    audio_features = get_audio_features(track['id'])
    print("Audio Features:", audio_features)

In [ ]:
import time

# Function to get audio features for a track with a delay
def get_audio_features_with_delay(track_id):
    time.sleep(1)  # Add a delay of 1 second between requests
    audio_features = sp.audio_features([track_id])[0]
    return audio_features

# Iterate over tracks in the playlist and get audio features for each track
num_tracks_to_process = 10  # Set the limit here
for index, item in enumerate(playlist_info['tracks']['items']):
    if index >= num_tracks_to_process:
        break

    track = item['track']
    print("Track Name:", track['name'])
    print("Artist(s):", ", ".join([artist['name'] for artist in track['artists']]))

    # Get audio features for the track with a delay
    audio_features = get_audio_features_with_delay(track['id'])
    print("Audio Features:", audio_features)


In [ ]:
#works but error reason: too many 429 error responses

import time

start_time = time.time()
artist_names = ['Drake', 'J Cole', 'Travis Scott']

all_tracks_info = []
all_audio_features = []

# Loop through each artist
for artist_name in artist_names:
    # Search for the artist
    results = sp.search(q='artist:' + artist_name, type='artist')
    artist_id = results['artists']['items'][0]['id']

    # Get albums from the artist
    albums = sp.artist_albums(artist_id, album_type='album', limit=50)

    # Iterate over each album
    for album in albums['items']:
        # Get the album ID
        album_id = album['id']

        # Get tracks from the album
        tracks = sp.album_tracks(album_id)

        # Iterate over each track
        for track in tracks['items']:
            # Get track information
            track_info = {
                'Artist Name': artist_name,
                'Track Name': track['name'],
                'Album Name': album['name'],
                'Release Date': album['release_date'],
            }
            all_tracks_info.append(track_info)

            # Get audio features for the track
            audio_features = sp.audio_features([track['id']])[0]
            all_audio_features.append(audio_features)

# Create DataFrames from the lists of track information and audio features
tracks_df = pd.DataFrame(all_tracks_info)
audio_features_df = pd.DataFrame(all_audio_features)

# Merge the two DataFrames on index
pd.concat([tracks_df, audio_features_df], axis=1)

end_time = time.time()

# Calculate runtime
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")

In [ ]:

import time

# Define a list of artist names
artist_names = ['Drake', 'J Cole', 'Travis Scott', 'Megan Thee Stallion', 'Doja Cat',
                'Kid Cudi', 'Nicki Minaj', '21 Savage', 'Metro Boomin', 'Gunna',
                'Kendrick Lamar', 'Jack Harlow', 'Eminem', 'Tupac', 'Biggie', 'NWA']

# Create empty lists to store track information and audio features
all_tracks_info = []
all_audio_features = []

# Loop through each artist
for artist_name in artist_names:
    retry_count = 0

    while True:
        try:
            # Search for the artist
            results = sp.search(q='artist:' + artist_name, type='artist')
            artist_id = results['artists']['items'][0]['id']

            # Get albums from the artist
            albums = sp.artist_albums(artist_id, album_type='album', limit=50)

            # Iterate over each album
            for album in albums['items']:
                # Get the album ID
                album_id = album['id']

                # Get tracks from the album
                tracks = sp.album_tracks(album_id)

                # Iterate over each track
                for track in tracks['items']:
                    # Get track information
                    track_info = {
                        'Artist Name': artist_name,
                        'Track Name': track['name'],
                        'Album Name': album['name'],
                        'Release Date': album['release_date'],
                    }
                    all_tracks_info.append(track_info)

                    # Get audio features for the track
                    audio_features = sp.audio_features([track['id']])[0]
                    all_audio_features.append(audio_features)

            # Break out of the retry loop if successful
            break

        except spotipy.SpotifyException as e:
            if e.http_status == 429:  # If rate limit exceeded
                if retry_count < 3:  # Retry up to 3 times
                    retry_count += 1
                    wait_time = 2 ** retry_count  # Exponential backoff
                    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Max retries reached. Exiting.")
                    break
            else:
                print("An error occurred:", e)
                break

# Create DataFrames from the lists of track information and audio features
tracks_df = pd.DataFrame(all_tracks_info)
audio_features_df = pd.DataFrame(all_audio_features)

# Merge the two DataFrames on index
pd.concat([tracks_df, audio_features_df], axis=1)
